In [ ]:
# Simple Example of Render Issue

In [ ]:
%%capture
!pip install spatialpandas

In [ ]:
import json
import spatialpandas as spd
import geopandas as gpd
import pandas as pd
import holoviews as hv
import geoviews as gv
import panel as pn
import cartopy.crs as ccrs
import datashader as ds

from holoviews import streams, opts
from bokeh.models import HoverTool
from holoviews.operation.datashader import (
    rasterize, shade, regrid, inspect_points,
    datashade, inspect_polygons
)
from shapely.geometry import Point

In [ ]:
# this is really read from a parquet file and has 10's of thousands of polygons
geojson = """
{
    "type": "FeatureCollection",
    "name": "polygons",
    "crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:EPSG::3857" } },
    "features": [
        { "type": "Feature", "properties": { "polygon_id": "0101", "name": "0101" }, "geometry": { "type": "Polygon", "coordinates": [ [ [ -8725286.109445322304964, 5274247.536411256529391 ], [ -8509674.627641066908836, 5062782.429257080890238 ], [ -8683822.362944504246116, 4801560.826301923021674 ], [ -8725286.109445322304964, 5274247.536411256529391 ] ] ] } },
        { "type": "Feature", "properties": { "polygon_id": "0102", "name": "0102" }, "geometry": { "type": "Polygon", "coordinates": [ [ [ -9156509.073053838685155, 4946683.939054788090289 ], [ -8895287.470098679885268, 4992294.06020568870008 ], [ -8775042.605246305465698, 4598388.468447910621762 ], [ -9135777.199803428724408, 4627413.09099848382175 ], [ -9156509.073053838685155, 4946683.939054788090289 ] ] ] } },
        { "type": "Feature", "properties": { "polygon_id": "1801", "name": "1801" }, "geometry": { "type": "Polygon", "coordinates": [ [ [ -13522641.579590054228902, 4822292.699552332051098 ], [ -13622154.571192018687725, 4627413.09099848382175 ], [ -13381664.841487269848585, 4511314.6007961910218 ], [ -13522641.579590054228902, 4822292.699552332051098 ] ] ] } },
        { "type": "Feature", "properties": { "polygon_id": "1802", "name": "1802" }, "geometry": { "type": "Polygon", "coordinates": [ [ [ -13203370.73153374902904, 4328874.116192588582635 ], [ -12958734.62717891857028, 4453265.355695044621825 ], [ -12784586.891875479370356, 4080091.637187676038593 ], [ -13203370.73153374902904, 4328874.116192588582635 ] ] ] } }
    ]
}
"""
geodict = json.loads(geojson)

In [ ]:
catchments_gdf = gpd.read_file(geojson)
catchments_gdf

In [ ]:
# this is queried from parquet files
metrics = pd.DataFrame(
    {"polygon_id": ["0101", "0102", "1801", "1802"],
    "bias": [1,2,4,3],
    "delta": [0.1, 0.14, 0.2, 0.125],
    "max": [100, 125, 200, 80]}
)
metrics

In [ ]:
def get_catchment_details(polygon_id:str):
    """This joins metrics to polygons"""
    
    # filter based on id
    filtered_metrics = metrics[metrics["polygon_id"].str.startswith(polygon_id)]
    
    # join metrics to polygons
    joined_gdf = catchments_gdf.merge(filtered_metrics, left_on="polygon_id", right_on="polygon_id")
    
    # convert to spatial pandas
    joined_spdf = spd.GeoDataFrame(joined_gdf)
        
    return joined_spdf

get_catchment_details("01")

In [ ]:
hv.extension('bokeh')

In [ ]:
opts = dict(
    width=700,
    height=500,
    show_grid=False
)

stream = streams.Tap()


def get_basemap():
    tiles = gv.tile_sources.OSM.opts(**opts)
    return tiles

    
def get_catchments(catchment_id, measure):
    spdf_catchments = get_catchment_details(catchment_id)

    catchments = gv.Polygons(
        spdf_catchments,
        crs=ccrs.GOOGLE_MERCATOR, 
        vdims=[measure, 'name', 'polygon_id']
    )
    
    # bounds = spd_map["geometry"].total_bounds    
    # catchments.opts(xlim=(bounds[0], bounds[1]), ylim=(bounds[2], bounds[3]))
    
    measure_min = spdf_catchments[measure].min()
    measure_max = spdf_catchments[measure].max()
    catchments = catchments.redim.range(**{f"{measure}": (measure_min, measure_max)})
    
    return catchments


def get_measure_selector():
    measures = [
            "bias",
            "max",
            "delta"
        ]  
    measure_selector = pn.widgets.Select(name='Measure', options=measures, value=measures[0], width_policy="fit") 
    return measure_selector


def get_cmap_selector():
    cmaps = odict([(n,colorcet.palette[n]) for n in ['fire', 'bgy', 'bgyw', 'bmy', 'gray', 'kbc']])
    cmap = pn.widgets.Select(name='Colormap', options=cmaps)
    return cmap


def get_aggregator(measure):
    return ds.mean(measure)


def get_huc_selector():
    hucs=[
        "01",
        "18",
    ]
    huc_selector = pn.widgets.Select(name='Region', options=hucs, value="01") 
    return huc_selector


def get_name_pane(x, y):
    if x is not None and y is not None:
        pnt = Point(x, y)
        catchment = catchments_gdf[(catchments_gdf.contains(pnt) == True)]
        catchment_name = catchment["name"].iloc[0]
        return pn.pane.HTML(f"{catchment_name}")
    return pn.pane.HTML(f"Select polygon to see name!")

In [ ]:
measure_selector = get_measure_selector()
huc_selector = get_huc_selector()

catchment_polygons = pn.bind(
    get_catchments, 
    catchment_id=huc_selector.param.value, 
    measure=measure_selector.param.value
)
aggregator = pn.bind(get_aggregator, measure=measure_selector.param.value)
raster_catchments = rasterize(hv.DynamicMap(catchment_polygons), aggregator=aggregator, precompute=True).opts(**opts, colorbar=True)

hover = inspect_polygons(raster_catchments).opts(fill_color='yellow', tools=["hover", 'tap']).opts(alpha=0.5)

stream.source = raster_catchments

layout = pn.Column(
    pn.Row(
        pn.pane.Markdown("# Minimum Example", width=800)
    ),
    pn.Row(
        pn.Column(
            measure_selector,
            huc_selector,
        ),
        pn.Column(
            pn.Row(
                hv.DynamicMap(get_basemap) * raster_catchments * hover
            ),
            pn.Row(
                pn.bind(get_name_pane, x=stream.param.x, y=stream.param.y)
            )
        )
    )
)
layout.servable()